In [1]:
!pip install pyLDAvis

In [2]:
import pandas as pd
import numpy as np

# NLTK
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import re

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib
%matplotlib inline
import seaborn as sns

# Model 1: Data Science Job Post

In [3]:
job_1 = 'Masters Degree in Operations Research, Applied Statistics, Data Mining, Machine Learning or a related quantitative \
discipline.Highly skilled at R Python programming for data science. Deep understanding of statistical and machine-learning \
approaches, including supervised (regression and classification techniques) and unsupervised (clustering) learning, as well as \
recommendation methods. Use of graphical packages, like ggplot2, matplotlib, plotly, or seaborn, to visualize outcomes to a wide \
audience. 4-5+ years’ experience delivering world-class data science outcomes, you solve complex analytical problems using quantitative \
approaches with your unique blend of analytical, mathematical and technical skills.Excellent communication skills, not only with \
other analytically-minded individuals, but a wider audience.Experience with cloud computing environments and \
tooling (e.g. AWS, Azure, Domino Data Lab). Familiarity with the software development lifecycle. Ability to independently navigate \
the data science process, ranging from identifying a business problem, acquiring data, experimenting with solutions, and sharing \
results. Familiarity and background in health care data analytics and use of protected health care data.'

job_2 = 'In pursuit of an undergraduate, graduate, or PhD degree at a four-year institution in Data Science, Math, Physics, Statistics,\
Computer Science,  Engineering, or related field.Deep desire to manipulate and analyze complex, high-volume, highly dimensional\
data from varying sources.Passion for empirical research and answering hard questions with data and models.Knowledge of statistical\
concepts such as correlation, linear regression, optimization, and how they can be applied.Exposure to statistical analysis\
tools/frameworks such as NumPy/SciPy/Pandas/SciKit learn, or SparkML.Expertise in Machine Learning/Cognitive Computing algorithms \
and techniques would be a bonus.Excellent problem solver with the ability to take initiative.Able to present findings and communicate \
effectively.Able to work collaboratively across teams and leverage constructive feedback.Naturally curious and skeptical.'

job_3 = 'Experience in data mining, data analysis, and predictive modelingPrevious work experience in business intelligence and \
data analytics Proven experience utilizing statistical techniques to identify key insights from data to solve business Problems'

job_4 = 'Work with a team of data scientists, machine learning engineers, software engineers and QA engineers.Perform data \
collection, preprocessing, feature engineering, data visualization and analysis.Build automation of data collection and \
preprocessing.Build models to address business problems.Engage with lines of business, users and analysts to explore and \
prototype opportunities and use-cases exploiting data and the application of cognitive and machine learning technologies.\
Design, develop, test and support Cognitive microservices to operationalize and productize deployment of resulting models and \
cognitive solutions.Modern, object-oriented or functional programming experience, (Java, C++, Python, Scala, SQL, R)\
Data Science and Machine Learning Frameworks (R, Apache Spark / MLlib, TensorFlow, Scikit-learn, etc…)Natural Language \
Processing (NLTK, CoreNLP, Gensim, Spacy, etc..)Experience with Big-Data technologies and cloud (AWS, other)Linux / Bash scripting\
Relational databases (Oracle, PostgreSQL, MySQL, etc.)Agile development methodologyCI/CD Development environments \
and tools (GIT, Maven, Jenkins, etc)RESTful Microservice APIsStrong analytical skills. Previous experience or education \
focused on statistics or data science is valuable.Bachelors degree in Computer Science or related degree.At least 1 year of \
experience through internships or field related work experience.Experience in working on an Open Source project a plus but \
not required.Good English language skillsAppreciate the value of diversity (in all its forms) brings to our team and companyPh.D. \
or M.S. in Computer Science, Statistics, or related field.Proficient with at least one major programming language (Java, Scala, \
C++, Python).Experience with supervised and unsupervised machine learning algorithms.Strong grasp of data structures and algorithms.\
Good communication skills with both technical and business people. Ability to speak at a level appropriate for the audience.Cares\
deeply about agile software practices, data driven development, and collaborating with other researchers and engineers.'

job_5 = 'Design and implement data analysis, data mining, research, analysis, modeling strategies and best practices Use analytical\
expertise to spot key business insights, trends, and opportunities in our vast database Help interpret and communicate your findings\
in a way that is easily understood by those less familiar with data Collaborate with a dynamic Product and Development team in delivering\
data products and insights to clients Establish and maintain relationships with distributed cross-functional teams to leverage domain and \
industry expertise Identify missing and inaccurate data in our platforms; seek out supplements and replacements to improve our data sets\
to the greatest extent possible Advise teams and clients as a subject matter expert in data science Bachelor’s degree in a quantitative\
field such as Statistics, Computer Science, Economics, or Mathematics; or equivalent work experience Relevant, successful professional \
industry experience Proficiency with database and analysis tools like R, SQL, Python, and others Expertise in statistical modeling, data\
collection, aggregation, analysis, machine learning, and productization Self-driven passion for finding and collaboratively solving \
problems Strong understanding of the benefits and flaws in the application of various methods'

job_6 = 'Bachelors or graduate degree in statistics/computer science or related field plus a minimum of 3 years applied analytics \
experience End-to-end experience developing approach/methodology given business need, ETL, data quality analysis, model development  \
and validation (machine learning or statistical), scoring, presentation of results and deployment into production environment\
(including AWS, Azure, Google Cloud) Strong experience with predictive models and clustering (segmentation) Strong/Expert SQL, \
Python, R, in a Hadoop environment H2O experience a plus Experience with Tableau, Birst, Microsoft Excel and PowerPoint Travel\
Requirement may be required, up to 20% Strong team player, flexible, and creative Experience with marketing related analytics\
channels/metrics, especially digital Passion/Curiosity around new techniques, ie deep learning Identity resolution experience, \
data with multiple levels and sources – aggregating & connecting Ability to learn and lead with minimal oversight, work on several \
projects at same time Strong communication and presentation skills, ability to maintain timeline'

## Step 1: Preprocess our text.

In [4]:
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
        1. Tokenizes and removes punctuation
        2. Removes stopwords
        3. Stems
        4. Returns a list of the cleaned text
    '''
    if pd.isnull(text):
        return []
    # tokenizing and removing punctuation
    tokenizer = RegexpTokenizer(r'\w+')
    text_processed=tokenizer.tokenize(text)
    
    # removing any stopwords
    text_processed = [word.lower() for word in text_processed if word.lower() not in stopwords.words('english')]
    
    # steming (commented out to get the non stemmed words)
#     porter_stemmer = PorterStemmer()
    
#     text_processed = [porter_stemmer.stem(word) for word in text_processed]
    
    try:
        text_processed.remove('b')
    except: 
        pass

    return text_processed ## <-- one small tweak

In [5]:
texts = [text_process(job_1),
         text_process(job_2),
         text_process(job_3),
         text_process(job_4),
         text_process(job_5),
        text_process(job_6)]

## Step 2: Fit LDA Model.

In [6]:
from gensim import corpora, models
# from scipy.special.logsumexp import 
import pyLDAvis.gensim

In [7]:
pyLDAvis.enable_notebook()

In [8]:
np.random.seed(42) # want to have reproducable resuslts

In [23]:
dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts] # bow = bag of words


# Instantiate the model
ldamodel = models.ldamodel.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                    num_topics=3,
                                   passes=10, # similar to epochs in neural networks
                                   minimum_probability=0)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

## Step 3: Visualize LDA model.

In [24]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      47.873772        1       1 -0.086036 -0.008413
1      30.540709        1       2  0.050762 -0.065894
0      21.585519        1       3  0.035274  0.074306, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
163   Default   4.000000      analysis   4.000000  30.0000  30.0000
171   Default   2.000000      insights   2.000000  29.0000  29.0000
6     Default  20.000000          data  20.000000  28.0000  28.0000
48    Default  14.000000    experience  14.000000  27.0000  27.0000
169   Default   1.000000      identify   1.000000  26.0000  26.0000
170   Default   1.000000           key   1.000000  25.0000  25.0000
287   Default   1.000000      database   1.000000  24.0000  24.0000
308   Default   1.000000      industry   1.000000  23.0000  23.0000
301   Default   1.000000       clients   1.000000  22.0000  22.0000
280   Default   1.000000      modeling   1.000000  21.0000  21.0000
7     Default   2.000000        mining   2.000000  20.0000  20.0000
140   Default   2.000000     expertise   2.000000  19.0000  19.0000
156   Default   2.000000         teams   2.000000  18.0000  18.0000
3     Default   2.000000      research   2.000000  17.0000  17.0000
88    Default   5.000000      business   5.000000  16.0000  16.0000
96    Default   1.000000        health   1.000000  15.0000  15.0000
69    Default   1.000000     computing   1.000000  14.0000  14.0000
11    Default   1.000000  quantitative   1.000000  13.0000  13.0000
53    Default   1.000000       complex   1.000000  12.0000  12.0000
62    Default   1.000000     excellent   1.000000  11.0000  11.0000
22    Default   1.000000    approaches   1.000000  10.0000  10.0000
42    Default   1.000000      outcomes   1.000000   9.0000   9.0000
97    Default   1.000000          care   1.000000   8.0000   8.0000
148   Default   1.000000          able   1.000000   7.0000   7.0000
13    Default   1.000000        highly   1.000000   6.0000   6.0000
25    Default   1.000000    regression   1.000000   5.0000   5.0000
89    Default   1.000000       problem   1.000000   4.0000   4.0000
78    Default   1.000000   familiarity   1.000000   3.0000   3.0000
21    Default   3.000000   statistical   3.000000   2.0000   2.0000
267   Default   4.000000        strong   4.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
339    Topic3   0.699494       solving   1.143140   1.0420  -5.3478
341    Topic3   0.699503         flaws   1.143155   1.0420  -5.3478
286    Topic3   0.699508          vast   1.143167   1.0420  -5.3478
279    Topic3   0.699492     implement   1.143154   1.0420  -5.3478
329    Topic3   0.699492      relevant   1.143157   1.0420  -5.3478
324    Topic3   0.699481      bachelor   1.143149   1.0419  -5.3478
326    Topic3   0.699484     economics   1.143168   1.0419  -5.3478
163    Topic3   2.800173      analysis   4.672550   1.0211  -3.9607
7      Topic3   1.225471        mining   2.280986   0.9119  -4.7871
156    Topic3   1.224692         teams   2.281361   0.9111  -4.7877
140    Topic3   1.224531     expertise   2.281350   0.9109  -4.7879
6      Topic3   5.442825          data  20.542239   0.2050  -3.2961
55     Topic3   1.226986      problems   2.992887   0.6415  -4.7859
88     Topic3   1.754805      business   5.655375   0.3629  -4.4281
48     Topic3   2.811292    experience  14.442214  -0.1034  -3.9568
21     Topic3   1.226962   statistical   3.604864   0.4554  -4.7859
153    Topic3   1.229861          work   4.416956   0.2546  -4.7835
18     Topic3   1.232533       science   8.288861  -0.3727  -4.7813
9      Topic3   0.705433      learning   7.863701  -0.8781  -5.3394
5      Topic3   0.704576    statistics   4.503016  -0.3218  -5.3406
109    Topic3   0.704327      computer   3.891210  -0.1761  -5.3409
80     Topic3  

## Step 4: Update model with new data!

In [25]:
job_7 = 'PhD degree in computer science, or related technical, math, or scientific field Strong working knowledge of \
deep learning, machine learning and statistics. Hands on experience building models with deep learning frameworks like \
MXNet, Tensorflow, Caffe, Torch, Theano or similar. Experience in using Python, R or Matlab or other statistical/machine \
learning software Strong communication and data presentation skills The motivation to achieve results in a fast-paced \
environment. Experience with statistical modeling / machine learning Strong attention to detail Comfortable working in a \
fast paced, highly collaborative, dynamic work environment Ability to think creatively and solve problems'

In [26]:
new_doc = text_process(job_7)

In [27]:
ldamodel.update([dictionary.doc2bow(new_doc)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [28]:
job_8 = 'ability to tell meaningful stories with data. Programming skills in Python Experience with SQL, database modeling and \
data warehousing principles. Creativity to go beyond current tools to deliver best solution to the problem. Data visualization \
experience in Matplotlib, Tableau, ggplot2 or a similar tool. Experience with data science algorithms including decision trees, \
probability networks, association rules, clustering, regression, and neural networks. Excellent interpersonal, written, and verbal \
communication skills. Ability and comfort working independently and making key decisions on projects. Build and monitor effective \
reporting and alerting for fraud and operational health of a line of business.  Perform ad-hoc and re-occurring statistical and data \
science analyses.  Work closely with data warehouse architects and software developers to generate seamless business intelligence \
solutions for end users.  Support production analytic solutions.  Present results of analyses to business units.'

In [29]:
new_doc2 = text_process(job_8)

In [30]:
ldamodel.update([dictionary.doc2bow(new_doc2)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [31]:
job_9 = 'Work in a collaborative, fast-paced environment with a team of smart, passionate developers, product managers, and designers\
 Rapidly iterate on a search and analytics SaaS product with a quickly growing number of users Own features and functionality \
 from design through deployment Design and develop algorithms, models, and pipelines that power the analytics of earned \
 media based on Machine Learning and Natural Language Processing (NLP) techniques Gain insights using data science techniques, \
 implement models, and engineer services to exploit those insights Practical experience at large scale in one or more of \
 the following domains: natural language processing, machine learning, search, information retrieval or text classification \
 Solid experience in Java, Python, Scala, or another object-oriented language Familiarity with ML/NLP frameworks such as \
 scikit-learn, Weka, Pandas, Mallet, TensorFlow, PyTorch, Theano Ability to thrive in a fast-paced, unstructured and dynamic \
 environment US Citizen or Permanent Resident (We will not sponsor at this time) Good understanding of system design, distributed \
 systems, cloud infrastructure, service-oriented solutions, scalable web services Familiarity with using clustered computing \
 systems, such as Hadoop/MapReduce, Spark/SparkR, Lucene/ElasticSearch, Storm, etc Familiarity with Agile development methodologies \
 (Kanban preferred)'

In [32]:
new_doc3 = text_process(job_9)

In [33]:
ldamodel.update([dictionary.doc2bow(new_doc3)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [35]:
job_10 = 'Use data science and machine learning to drive product innovation, customer success, marketing optimization, and more\
Help champion a data-driven culture and push long-term business value creation through development of best-in-class data science \
capabilities Leverage data science, computer vision, and machine learning to increase value across our businesses Train computer \
vision and deep learning models on the big data resources Optimize computer vision algorithms and deep learning models to run \
efficiently in production system Work with engineers from prototype to production Minimum of M.S. in Computer Science, Math, Physics, \
Engineering, Statistics or other technical fields. 2+ years working experience in machine learning / computer vision / deep learning. \
Expert level in machine learning, computer vision, and deep learning. Proven track record of completing multiple data science projects \
end to end, from idea generation to objectives formulation to implementation and deliverables. Proficiency in C/C++, Java, SQL, Python, \
and multi-threaded programming. Experience with Hadoop stack (HIVE, Pig, Hadoop streaming), MapReduce and SPARK. Experience with deep \
learning tools (tensorFlow, caffe(2)).  Excellent written and oral communication skills Highly skilled at translating business \
requirements from key stakeholders into shovel-ready work plans Excellent project management skills, with a proven track record of \
on-time and within-budget delivery'

In [36]:
new_doc4 = text_process(job_10)

In [37]:
ldamodel.update([dictionary.doc2bow(new_doc4)])

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/gensim/models/ldamodel.py:776: DeprecationWarning: `logsumexp` is deprecated!
Importing `logsumexp` from scipy.misc is deprecated in scipy 1.0.0. Use `scipy.special.logsumexp` instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elog

In [38]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/Users/guillermogonzalez/miniconda3/envs/ga/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      50.210454        1       1 -0.099004  0.013427
2      29.853135        1       2  0.200540 -0.000113
0      19.936411        1       3 -0.101536 -0.013315, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
9     Default  20.000000       learning  20.000000  30.0000  30.0000
6     Default  12.000000           data  12.000000  29.0000  29.0000
109   Default  12.000000       computer  12.000000  28.0000  28.0000
18    Default  10.000000        science  10.000000  27.0000  27.0000
19    Default  10.000000           deep  10.000000  26.0000  26.0000
8     Default  11.000000        machine  11.000000  25.0000  25.0000
48    Default   7.000000     experience   7.000000  24.0000  24.0000
61    Default   4.000000         skills   4.000000  23.0000  23.0000
209   Default   4.000000              c   4.000000  22.0000  22.0000
259   Default   4.000000          value   4.000000  21.0000  21.0000
167   Default   4.000000         proven   4.000000  20.0000  20.0000
345   Default   4.000000            end   4.000000  19.0000  19.0000
357   Default   4.000000     production   4.000000  18.0000  18.0000
62    Default   4.000000      excellent   4.000000  17.0000  17.0000
88    Default   4.000000       business   4.000000  16.0000  16.0000
153   Default   5.000000           work   5.000000  15.0000  15.0000
361   Default   6.000000         hadoop   6.000000  14.0000  14.0000
125   Default   7.000000         models   7.000000  13.0000  13.0000
78    Default   3.000000    familiarity   3.000000  12.0000  12.0000
195   Default   3.000000         design   3.000000  11.0000  11.0000
218   Default   3.000000       language   3.000000  10.0000  10.0000
250   Default   2.000000        working   2.000000   9.0000   9.0000
63    Default   2.000000  communication   2.000000   8.0000   8.0000
17    Default   3.000000    programming   3.000000   7.0000   7.0000
211   Default   3.000000            sql   3.000000   6.0000   6.0000
393   Default   3.000000       projects   3.000000   5.0000   5.0000
170   Default   3.000000            key   3.000000   4.0000   4.0000
235   Default   3.000000          tools   3.000000   3.0000   3.0000
282   Default   3.000000           best   3.000000   2.0000   2.0000
5     Default   2.000000     statistics   2.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
62     Topic3   0.634385      excellent   4.950471  -0.4420  -5.3660
357    Topic3   0.612942     production   4.932359  -0.4727  -5.4004
170    Topic3   0.499985            key   3.035369  -0.1909  -5.6041
282    Topic3   0.495188           best   3.031400  -0.1992  -5.6138
345    Topic3   0.592785            end   4.915592  -0.5027  -5.4339
235    Topic3   0.487143          tools   3.024668  -0.2134  -5.6301
211    Topic3   0.478553            sql   3.017493  -0.2288  -5.6479
393    Topic3   0.477814       projects   3.016938  -0.2302  -5.6495
17     Topic3   0.467823    programming   3.008674  -0.2485  -5.6706
109    Topic3   0.728836       computer  12.326992  -1.2155  -5.2272
19     Topic3   0.642009           deep  10.431784  -1.1754  -5.3541
94     Topic3   0.329367        results   1.090031   0.4158  -6.0215
167    Topic3   0.471963         proven   4.844308  -0.7160  -5.6618
259    Topic3   0.471863          value   4.844219  -0.7162  -5.6620
209    Topic3   0.471816              c   4.844165  -0.7163  -5.6621
332    Topic3   0.400526    proficiency   2.965591  -0.3894  -5.8259
157    Topic3   0.400524       leverage   2.965607  -0.3894  -5.8259
323    Topic3   0.400503         expert   2.965565  -0.3895  -5.8260
276    Topic3   0.400503         driven   2.965565  -0.3895  -5.8260
33     Topic3   0.400499            use   2.965596  -0.3895  -5.8260
108    Topic3   0.400497      

In [68]:
# TOPIC 1
ldamodel.print_topics(num_topics=3)

[(0,
  '0.005*"computer" + 0.005*"business" + 0.005*"deep" + 0.005*"excellent" + 0.005*"production" + 0.004*"end" + 0.004*"key" + 0.004*"best" + 0.004*"tools" + 0.004*"sql"'),
 (1,
  '0.013*"learning" + 0.010*"machine" + 0.010*"familiarity" + 0.010*"design" + 0.010*"language" + 0.010*"models" + 0.009*"product" + 0.008*"hadoop" + 0.007*"techniques" + 0.007*"analytics"'),
 (2,
  '0.079*"learning" + 0.054*"data" + 0.053*"computer" + 0.045*"science" + 0.044*"deep" + 0.035*"machine" + 0.027*"experience" + 0.018*"work" + 0.018*"skills" + 0.018*"c"')]

In [66]:
ldamodel.print_topic(topicno=1)

'0.013*"learning" + 0.010*"machine" + 0.010*"familiarity" + 0.010*"design" + 0.010*"language" + 0.010*"models" + 0.009*"product" + 0.008*"hadoop" + 0.007*"techniques" + 0.007*"analytics"'